# Problem Setup and Configuration
We consider the problem of simple harmonic motion with the governing equation for displacement:

$$x = A \cos(\omega t + \phi)$$
$$ \omega = sqrt(k/m)$$


In [ ]:
import math, torch
import data
import model
import train
import evaluation
from config import Config

def to_dtype(s): return {"float32": torch.float32, "float64": torch.float64}[s]

if __name__ == "__main__":
    cfg = Config()                               # 1) pick defaults
    device = cfg.device
    dtype  = to_dtype(cfg.dtype_str)

    # 2) make dataset
    t_data, x_data, _ = data.data_generation(cfg, device, dtype)

    # 3) compute initial conditions from (A, phi) -> (x0, v0)
    w = math.sqrt(cfg.k/cfg.m)
    x0 = cfg.A * math.cos(cfg.phi)
    v0 = -cfg.A * w * math.sin(cfg.phi)

    # 4) build model
    net = model.build_model(cfg, device, dtype, x0, v0)

    # 5) train
    net = train.train(net, cfg, t_data, x_data, device, dtype)

    # 6) evaluate / plot
    evaluation.evaluate_and_plot(net, cfg, t_data, x_data)


ModuleNotFoundError: No module named 'torch'

# Data Generation
- A time vector is created between t_start and t_end, with N evenly spaced points.
- An exact solution is computed from the equation above.
- Gaussian noise with standard deviation noise_std is added to simulate sensor noise.

## Notes:
- omega is calculated here
- 

In [ ]:

import math, numpy as np, torch

def omega(cfg): 
    # Calculate angular frequency for use in exact solution
    return math.sqrt(cfg.k / cfg.m)

def true_x(cfg, t_np: np.ndarray) -> np.ndarray:
    # Exact solution for simple harmonic motion
    return cfg.A * np.cos(omega(cfg) * t_np + cfg.phi)

def data_generation(cfg, device, dtype):
    N = getattr(cfg, "N_data", 200)
    seed = getattr(cfg, "seed", None)
    if seed is not None:
        np.random.seed(seed)     # !!!!!!!!!!! controls NumPy's random numbers (noise)
        torch.manual_seed(seed)  # !!!!!!!!!!! controls PyTorch RNG (not strictly needed here, but good practice)
    t = np.linspace(cfg.t_start, cfg.t_end, N)  # NumPy time vector
    noise = cfg.noise_std * np.random.randn(len(t)) # ... if cfg.use_data_loss else 0.0
    x_clean = true_x(cfg, t)    # analytical solution
    x_noisy = x_clean + noise   # noisy measurements
    t_data = torch.tensor(t, device=device, dtype=dtype).view(-1,1)     # time vector in tensor form
    x_data = torch.tensor(x_noisy, device=device, dtype=dtype).view(-1,1)   # noisy measurements in tensor form
    return t_data, x_data, x_clean

# PINN Model
Next, a standard multilayer perceptron (MLP) is constructed based on previously defined hyperparameters.
- fully connected neural net (MLP)
- input = time; output = displacement prediction x(t)
- this is where the hyperparameters come into play (activation functions, network depth and width, init (xavier))
- 

In [ ]:
import torch, torch.nn as nn

def get_activation(name):                                                       # get activation function by name from config
    return {"tanh": nn.Tanh(), "relu": nn.ReLU(), "silu": nn.SiLU()}[name]      # table/dictionary lookup

def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)                                       # Xavier initialization to weights ********* add he
        nn.init.zeros_(m.bias)                                                  # set biases to zero

class MLP(nn.Module):
    def __init__(self, cfg, x0, v0, device, dtype):
        super().__init__()

        self.hard_ics = cfg.hard_ics                                        # boolean, enforching hard ics or not
        self.x0 = torch.tensor([[x0]], device=device, dtype=dtype)
        self.v0 = torch.tensor([[v0]], device=device, dtype=dtype)          # !!!!!!!!!!! initial position and velocity as tensors?

        act = get_activation(cfg.activation)

        layers = [nn.Linear(1, cfg.hidden_width), act]                      # input layer 
        for _ in range(cfg.hidden_layers-1):                                # hidden layers
            layers += [nn.Linear(cfg.hidden_width, cfg.hidden_width), act]
        layers += [nn.Linear(cfg.hidden_width, 1)]                          # output layer
        self.net = nn.Sequential(*layers)         
        if cfg.init == "xavier": self.net.apply(xavier_init)                # apply to submodules

    def forward(self, t):
        raw = self.net(t)                                                       
        return self.x0 + self.v0*t + (t**2)*raw if self.hard_ics else raw   # enforce hard ics if specified     

def build_model(cfg, device, dtype, x0, v0):                                     
    return MLP(cfg, x0, v0, device, dtype).to(device).to(dtype)             # redundant to .to(dtype) ?

# Training
Physics constraints are integrated into the training function by incorporating a loss function for initial conditions and governing equations alongside the standard data loss.
- 

In [ ]:
from time import time
import torch, torch.nn as nn

def ddt(y, t):      # first derivative of *******y wrt t, returns gradient, creates graph for higher derivatives
    return torch.autograd.grad(y, t, torch.ones_like(y), create_graph=True)[0]

    # return torch.autograd.grad(
    #     outputs=y, inputs=t,
    #     grad_outputs=torch.ones_like(y),
    #     create_graph=True
    # )[0]

def d2dt2(y, t):    # second derivative of y wrt t (repeat ddt())
    return ddt(ddt(y, t), t)

_MSE = nn.MSELoss()

# !!!!!! collocation sampling, check with original
def sample_times(cfg, device, dtype):
    t = torch.rand(cfg.n_phys_per_step,1, device=device, dtype=dtype)*(cfg.t_end-cfg.t_start)+cfg.t_start
    t.requires_grad_(True)
    return t

# ---- loss functions ----
def physics_loss(model, cfg, t_phys):
    x = model(t_phys)
    x_tt = d2dt2(x, t_phys)
    return _MSE(x_tt + (cfg.k/cfg.m)*x, torch.zeros_like(x))

def ic_loss(model, cfg, device, dtype):
    if getattr(model, "hard_ics", False):
        return torch.tensor(0.0, device=device, dtype=dtype)
    t0 = torch.zeros(1,1, device=device, dtype=dtype).requires_grad_(True)
    x0 = model.x0; v0 = model.v0
    x_pred = model(t0); v_pred = ddt(x_pred, t0)
    return _MSE(x_pred, x0) + _MSE(v_pred, v0)

def data_loss(model, t_data, x_data):
    return _MSE(model(t_data), x_data)

# ---- training loop ----
def train(model, cfg, t_data, x_data, device, dtype):
    opt = torch.optim.Adam(model.parameters(), lr=cfg.lr)                 # !!!! Optimised - make configurable?
    
    curve_total, curve_phys, curve_ic, curve_data = [], [], [], []
    t0 = time.time()  # start timer to log train_time_min

    for step in range(1, cfg.steps+1):
        opt.zero_grad(set_to_none=True)

        t_phys = sample_times(cfg, device, dtype)
        l_phys = physics_loss(model, cfg, t_phys)
        l_ic   = ic_loss(model, cfg, device, dtype)
        l_data = data_loss(model, t_data, x_data) if cfg.use_data_loss else torch.tensor(0.0, device=device, dtype=dtype)
        loss = cfg.w_phys*l_phys + cfg.w_ic*l_ic + cfg.w_data*l_data # total loss

        loss.backward(); opt.step() # backprop using previously defined optimiser
        
        # logging
        curve_total.append(loss.item())
        curve_phys.append(l_phys.item())
        curve_ic.append(l_ic.item())
        curve_data.append(l_data.item())


        if step % cfg.print_every == 0:
            print(f"[{step:4d}] total={loss.item():.3e} phys={l_phys.item():.3e} ic={l_ic.item():.3e} data={l_data.item():.3e}")
        # End stats
        train_minutes = (time.time() - t0) / 60.0
        stats = {
            "final_total": curve_total[-1],
            "final_phys":  curve_phys[-1],
            "final_ic":    curve_ic[-1],
            "final_data":  curve_data[-1],
            "curve_total": curve_total,
            "curve_phys":  curve_phys,
            "curve_ic":    curve_ic,
            "curve_data":  curve_data,
            "train_time_min": train_minutes,
        }
    return model, stats

# Evaluation

In [ ]:
import numpy as np, torch
from data import true_x

@torch.no_grad()
def evaluate_and_plot(model, cfg, t_data, x_data):
    t_eval = torch.linspace(cfg.t_start, cfg.t_end, 801, device=cfg.device, dtype={"float32":torch.float32,"float64":torch.float64}[cfg.dtype_str]).unsqueeze(1)
    x_pred = model(t_eval).cpu().numpy().squeeze()
    x_true = true_x(cfg, t_eval.cpu().numpy().squeeze())
    l2 = float(np.sqrt(np.mean((x_pred - x_true)**2)))
    print(f"[Eval] L2 error = {l2:.3e}")
    try:
        import matplotlib.pyplot as plt
        plt.figure(); plt.scatter(t_data.cpu(), x_data.cpu(), s=10, c='tab:red', label='data')
        plt.plot(t_eval.cpu(), torch.tensor(x_true), 'k--', label='exact')
        plt.plot(t_eval.cpu(), torch.tensor(x_pred), label='PINN')
        plt.legend(); plt.tight_layout(); plt.show()
    except Exception:
        pass
    return l2
